In [ ]:
import requests
import pandas as pd
import json
import ast

class readURL:
    data: pd.DataFrame

    def __init__(self, url: str):
        self.url = url
        
    def read(self):
        payload = ""
        headers = {}
        response = requests.request("GET", self.url, headers=headers, data=payload)
        return response
    
    def text(self):
        read = self.read().text
        return read

    def only_dict(self, d):
        return ast.literal_eval(d)

    def list_of_dicts(self, ld):
        return dict([(list(d.values())[1], list(d.values())[0]) for d in ast.literal_eval(ld)])

    def expand(self, withId:bool = True) -> pd.DataFrame:
        collumns = self.data.columns
        for col in collumns:
            if self.data[col].dtype == object and self.data[col].name != '_id' :
                #if withId:
                newColumn = self.data[col].tolist()
                print(newColumn)
                #newColumn = pd.json_normalize(newColumn).add_prefix(str(self.data[col].name) + '.')
                #else:
                    #newColumn = pd.json_normalize(data[x].apply(ast.literal_eval).tolist()).drop(columns=['_id']).add_prefix(str(data[x].name) + '.')
                #data = data.drop(columns=data[x].name)
                #data = data.join([newColumn])
        return self.data

    def dataFrame(self, withId:bool = True, columnExpand:bool = True) -> pd.DataFrame:
        self.data = pd.read_json(self.text())
        if not withId:
            self.data = self.data.drop(columns=['_id'])
        if columnExpand:
            self.data = self.expand(withId=withId)
        return self.data
  

In [ ]:
from pymongo import MongoClient
import pymongo
from django.http.response import JsonResponse
from bson.json_util import dumps, loads

# Requires the PyMongo package.
# https://api.mongodb.com/python/current

client = MongoClient('mongodb://localhost:27017/')
result = client['CeDRI_UGV']['ActionsApplication_queue'].aggregate([
    {
        '$sort': {
            'dateTime': -1
        }
    }, {
        '$limit': 1
    }
])

list(result)[0]

In [ ]:
battery = readURL("http://127.0.0.1:8000/battery/battery")
df = battery.dataFrame(withId=False)
df

In [ ]:

import requests
import pandas as pd
import json
import ast

class readJSON:
    def __init__(self, json, pandas: pd.DataFrame):
        self.json = json
        if pandas == None:
            self.pandas = pd.read_json(self.json)

    def only_dict(self, d):
        return ast.literal_eval(d)

    def list_of_dicts(self, ld):
        return dict([(list(d.values())[1], list(d.values())[0]) for d in ast.literal_eval(ld)])

    def expand(self, withId:bool = True) -> pd.DataFrame:
        data = self.pandas
        for x in data.columns:
            if data[x].dtypes == object and data[x].name != '_id' :
                if withId:
                    newColumn = pd.json_normalize(data[x].apply(ast.literal_eval).tolist()).add_prefix(str(data[x].name) + '.')
                else:
                    newColumn = pd.json_normalize(data[x].apply(ast.literal_eval).tolist()).drop(columns=['_id']).add_prefix(str(data[x].name) + '.')
                data = data.drop(columns=data[x].name)
                data = data.join([newColumn])
        return data

    def dataFrame(self, withId:bool = True, columnExpand:bool = True) -> pd.DataFrame:
        data = self.pandas
        if not withId:
            self.pandas = self.pandas.drop(columns=['_id'])
        if columnExpand:
            self.pandas = self.expand(withId=withId)
        self.json = data.to_json()
        return data

In [ ]:
json_ = readJSON(battery.json())
json_.dataFrame(withId=False)

In [ ]:
import requests
import json
import bson

url = "http://127.0.0.1:8000/actions/queue"
url1 = "http://127.0.0.1:8000/actions/actions/1"

_json = {
  "QueueNumber": "2",
  "Action": bson.ObjectId(oid='63946e374bb315a0e907d6dc')
}
#payload = json.dumps(_json)
headers = {
  'Content-Type': 'application/json'
}

#response = requests.request("Ge", url, headers=headers, data=payload)
response = requests.request("GET", url1)

response.text




In [ ]:
import datetime
_bson = {
  "QueueNumber": "2",
  "Action": bson.ObjectId(oid='63946e374bb315a0e907d6dc')
}
_json

In [18]:
from pymongo import MongoClient
from random import randint, random
import math
import datetime

# Requires the PyMongo package.
# https://api.mongodb.com/python/current

client = MongoClient('mongodb://localhost:27017/')['CeDRI_UGV']['BatteryApplication_battery']


stime = datetime.datetime(year=2022, month=12, day=16, hour=0, minute=0, second=0)

charge = 0.5

for j in range(0,100):
    print(stime)
    target = random()
    direction = ((target - charge) < 0)*-2 + 1
    while ((target - charge)*direction) > 0:
        charge += 0.01*direction*random() if direction < 0 else 0.005
        stime = stime.__add__(datetime.timedelta(minutes=1))
        _bson = {
            "dateTime": stime,
            "Calculate": {
                "Percent": round(charge,3),
                "Status": 'C' if direction > 0 else 'D' 
            },
            "Sensor":{
                "Voltage": random()*25,
                "Current": random()*5,
                "Temperature": random()*50
            }
        }
        #print(_bson)
        try:
            result = client.insert_one(_bson)
            #print(result)
        except:
            print('Fail')
    for i in range(0,randint(3,500)):
        if(charge < 0.01):
            break
        stime = stime.__add__(datetime.timedelta(minutes=1))
        charge -= 0.00001
        _bson = {
            "dateTime": stime,
            "Calculate": {
                "Percent": round(charge,3),
                "Status": 'C' if direction > 0 else 'D' 
            },
            "Sensor":{
                "Voltage": random()*25,
                "Current": random()*5,
                "Temperature": random()*50
            }
        }
        #print(_bson)
        try:
            result = client.insert_one(_bson)
            #print(result)
        except:
            print('Fail')



2022-12-16 00:00:00
2022-12-16 04:48:00
2022-12-16 05:52:00
2022-12-16 08:50:00
2022-12-16 16:00:00
2022-12-16 22:11:00
2022-12-17 04:31:00
2022-12-17 10:24:00
2022-12-17 12:31:00
2022-12-17 20:41:00
2022-12-17 22:16:00
2022-12-18 03:27:00
2022-12-18 08:54:00
2022-12-18 13:55:00
2022-12-18 17:33:00
2022-12-18 23:25:00
2022-12-19 05:33:00
2022-12-19 13:46:00
2022-12-19 22:10:00
2022-12-20 02:52:00
2022-12-20 09:03:00
2022-12-20 12:39:00
2022-12-20 19:11:00
2022-12-20 21:10:00
2022-12-21 03:46:00
2022-12-21 13:43:00
2022-12-21 15:25:00
2022-12-21 18:24:00
2022-12-21 21:04:00
2022-12-22 00:10:00
2022-12-22 02:40:00
2022-12-22 04:02:00
2022-12-22 09:18:00
2022-12-22 15:59:00
2022-12-22 17:20:00
2022-12-23 01:21:00
2022-12-23 04:33:00
2022-12-23 13:10:00
2022-12-23 19:06:00
2022-12-23 22:45:00
2022-12-24 01:51:00
2022-12-24 07:09:00
2022-12-24 11:59:00
2022-12-24 17:52:00
2022-12-24 23:28:00
2022-12-25 00:40:00
2022-12-25 03:06:00
2022-12-25 10:58:00
2022-12-25 20:00:00
2022-12-25 21:02:00


In [17]:
from pymongo import MongoClient
import bson
from random import random

# Requires the PyMongo package.
# https://api.mongodb.com/python/current

client = MongoClient('mongodb://localhost:27017/')
result = client['CeDRI_UGV']['BatteryApplication_battery'].aggregate([
    {
        '$limit': 1
    }
])
result = client['CeDRI_UGV']['BatteryApplication_battery'].delete_many({})

try:
    #result = client['CeDRI_UGV']['RouteApplication_route'].insert_one(_bson)
    print(result)
    #print(list(result))
except:
    print('Fail')

In [ ]:
a = datetime.datetime(year=2022, month=12, day=16, hour=int((i/size)*24), minute=int((i/size)*60), second=int((i/size)*60))
a = a.__add__(datetime.timedelta(days=0.78655))
print(a)